In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import klib

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score, classification_report
import warnings
warnings.filterwarnings('ignore')
from tabulate import tabulate

In [3]:
df = pd.read_csv(r"C:\Users\Saidabrorkhon\ML_Lectures\global_shark_attacks.csv")

In [3]:
df.head()

,date,year,type,country,area,location,activity,name,sex,age,fatal_y_n,time,species
0,2023-05-13,2023.0,Unprovoked,AUSTRALIA,South Australia,Elliston,Surfing,Simon Baccanello,M,46,Y,10h10,White shark
1,2023-04-29,2023.0,Unprovoked,AUSTRALIA,Western Australia,"Yallingup, Busselton",Swimming,male,M,NaN,N,11h20,1m shark
2,2022-10-07,2022.0,Unprovoked,AUSTRALIA,Western Australia,Port Hedland,Spearfishing,Robbie Peck,M,38,N,11h30,Bull shark
3,2021-10-04,2021.0,Unprovoked,USA,Florida,"Fort Pierce State Park, St. Lucie County",Surfing,Truman Van Patrick,M,25.0,N,NaN,NaN
4,2021-10-03,2021.0,Unprovoked,USA,Florida,"Jensen Beach, Martin County",Swimming,male,M,NaN,N,12h00,NaN


In [4]:
df.isnull().sum()

date          303
year          132
type           19
country        51
area          481
location      565
activity      586
name          220
sex           572
age          2987
fatal_y_n       0
time         3518
species      3118
dtype: int64

In [5]:
df.shape

(6890, 13)

In [4]:
df = klib.data_cleaning(df)

Shape of cleaned data: (6886, 13) - Remaining NAs: 12531


Dropped rows: 4
     of which 3 duplicates. (Rows (first 150 shown): [3923, 5356, 6370])

Dropped columns: 0
     of which 0 single valued.     Columns: []
Dropped missing values: 21
Reduced memory by at least: 0.16 MB (-23.53%)



In [12]:
df.shape

(6886, 13)

In [7]:
df.isnull().sum()

date          301
year          131
type           18
country        50
area          480
location      564
activity      585
name          219
sex           571
age          2983
fatal_y_n       0
time         3514
species      3115
dtype: int64

In [ ]:
df.info() # --> klib

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6886 entries, 0 to 6885
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   date       6585 non-null   string  
 1   year       6755 non-null   float32 
 2   type       6868 non-null   category
 3   country    6836 non-null   string  
 4   area       6406 non-null   string  
 5   location   6322 non-null   string  
 6   activity   6301 non-null   string  
 7   name       6667 non-null   string  
 8   sex        6315 non-null   category
 9   age        3903 non-null   string  
 10  fatal_y_n  6886 non-null   category
 11  time       3372 non-null   string  
 12  species    3771 non-null   string  
dtypes: category(3), float32(1), string(9)
memory usage: 532.3 KB


In [ ]:
df.info() # --> no klib (look at the memory usage)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6890 entries, 0 to 6889
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       6587 non-null   object 
 1   year       6758 non-null   float64
 2   type       6871 non-null   object 
 3   country    6839 non-null   object 
 4   area       6409 non-null   object 
 5   location   6325 non-null   object 
 6   activity   6304 non-null   object 
 7   name       6670 non-null   object 
 8   sex        6318 non-null   object 
 9   age        3903 non-null   object 
 10  fatal_y_n  6890 non-null   object 
 11  time       3372 non-null   object 
 12  species    3772 non-null   object 
dtypes: float64(1), object(12)
memory usage: 699.9+ KB


Data Preprocessing

In [5]:
#For klib

for col in df.columns:
  if df[col].dtype == "string" or df[col].dtype.name == "category":
    df[col].fillna(df[col].mode()[0], inplace=True)
  elif df[col].dtype in ['float32', 'int32']:
    df[col].fillna(df[col].median(), inplace=True)

In [6]:
# no klib

for col in df.columns:
  if df[col].dtype == "object":
    df[col].fillna(df[col].mode()[0], inplace=True)
  elif df[col].dtype in ['float64', 'int64']:
    df[col].fillna(df[col].median(), inplace=True)

In [7]:
df['year']

0       2023.0
1       2023.0
2       2022.0
3       2021.0
4       2021.0
         ...  
6885    1986.0
6886    1986.0
6887    1986.0
6888    1986.0
6889    1986.0
Name: year, Length: 6890, dtype: float64

In [6]:
df.isnull().sum()

date         0
year         0
type         0
country      0
area         0
location     0
activity     0
name         0
sex          0
age          0
fatal_y_n    0
time         0
species      0
dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6886 entries, 0 to 6885
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   date       6886 non-null   string  
 1   year       6886 non-null   float32 
 2   type       6886 non-null   category
 3   country    6886 non-null   string  
 4   area       6886 non-null   string  
 5   location   6886 non-null   string  
 6   activity   6886 non-null   string  
 7   name       6886 non-null   string  
 8   sex        6886 non-null   category
 9   age        6886 non-null   string  
 10  fatal_y_n  6886 non-null   category
 11  time       6886 non-null   string  
 12  species    6886 non-null   string  
dtypes: category(3), float32(1), string(9)
memory usage: 532.3 KB


Encoding

In [6]:
df_encoded = df.copy()

# for klib
for col in df_encoded.select_dtypes(include=['string','category']).columns:
  cardinality = df_encoded[col].nunique()
  if cardinality >= 5:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df_encoded[col])
  else:
    df_encoded = pd.get_dummies(df_encoded, columns=[col], dtype=int, drop_first=True) 

 

In [14]:
# for no klib

df_encoded = df.copy()
for col in df_encoded:
  if df_encoded[col].dtypes =='object':
   cardinality = df_encoded[col].nunique()
  if cardinality >= 5:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df_encoded[col])
  else:
    df_encoded = pd.get_dummies(df_encoded, columns=[col], dtype=int, drop_first=True) 

In [11]:
df_encoded.head()

,date,year,type,country,area,location,activity,name,sex,age,fatal_y_n,time,species
0,5534,2023.0,8,12,707,1124,1165,4662,2,122,5,92,1289
1,5529,2023.0,8,12,841,4378,1201,5511,2,24,2,122,165
2,5492,2022.0,8,12,839,3198,1054,4290,2,102,2,125,641
3,5413,2021.0,8,208,261,1257,1165,4998,2,59,2,341,1289
4,5412,2021.0,8,208,261,1686,1201,5511,2,24,2,140,1289


In [15]:
df = df_encoded

In [13]:
df.head()

,date,year,type,country,area,location,activity,name,sex,age,fatal_y_n,time,species
0,5534,2023.0,8,12,707,1124,1165,4662,2,122,5,92,1289
1,5529,2023.0,8,12,841,4378,1201,5511,2,24,2,122,165
2,5492,2022.0,8,12,839,3198,1054,4290,2,102,2,125,641
3,5413,2021.0,8,208,261,1257,1165,4998,2,59,2,341,1289
4,5412,2021.0,8,208,261,1686,1201,5511,2,24,2,140,1289


In [ ]:
df.info() #klib

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6886 entries, 0 to 6885
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       6886 non-null   int64  
 1   year       6886 non-null   float32
 2   type       6886 non-null   int64  
 3   country    6886 non-null   int64  
 4   area       6886 non-null   int64  
 5   location   6886 non-null   int64  
 6   activity   6886 non-null   int64  
 7   name       6886 non-null   int64  
 8   sex        6886 non-null   int64  
 9   age        6886 non-null   int64  
 10  fatal_y_n  6886 non-null   int64  
 11  time       6886 non-null   int64  
 12  species    6886 non-null   int64  
dtypes: float32(1), int64(12)
memory usage: 672.6 KB


In [ ]:
df.info() #no klib

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6890 entries, 0 to 6889
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   date       6890 non-null   int64
 1   year       6890 non-null   int64
 2   type       6890 non-null   int64
 3   country    6890 non-null   int64
 4   area       6890 non-null   int64
 5   location   6890 non-null   int64
 6   activity   6890 non-null   int64
 7   name       6890 non-null   int64
 8   sex        6890 non-null   int64
 9   age        6890 non-null   int64
 10  fatal_y_n  6890 non-null   int64
 11  time       6890 non-null   int64
 12  species    6890 non-null   int64
dtypes: int64(13)
memory usage: 699.9 KB


Scaling

In [7]:
num_col = df.select_dtypes(include=['int64', 'float32']).columns

#klib
scaler = StandardScaler()
df[num_col] = scaler.fit_transform(df[num_col])
df.head()

,date,year,type,country,area,location,activity,name,sex,age,fatal_y_n,time,species
0,2023-05-13,0.928780,Unprovoked,AUSTRALIA,South Australia,Elliston,Surfing,Simon Baccanello,M,46,Y,10h10,White shark
1,2023-04-29,0.928780,Unprovoked,AUSTRALIA,Western Australia,"Yallingup, Busselton",Swimming,male,M,16.0,N,11h20,1m shark
2,2022-10-07,0.910832,Unprovoked,AUSTRALIA,Western Australia,Port Hedland,Spearfishing,Robbie Peck,M,38,N,11h30,Bull shark
3,2021-10-04,0.892885,Unprovoked,USA,Florida,"Fort Pierce State Park, St. Lucie County",Surfing,Truman Van Patrick,M,25.0,N,Afternoon,White shark
4,2021-10-03,0.892885,Unprovoked,USA,Florida,"Jensen Beach, Martin County",Swimming,male,M,16.0,N,12h00,White shark


In [17]:
#no klib
num_col = df.select_dtypes(include=['int64']).columns
scaler = StandardScaler()
df[num_col] = scaler.fit_transform(df[num_col])

In [18]:
# choosing linear assuming our target variable is species
df['species'].nunique()

1560

In [22]:
df['species'].value_counts()

species
 0.661725    3307
 0.165249     105
-0.554035     102
 0.131344      92
 0.375949      89
             ... 
-1.803700       1
-0.096308       1
 0.608445       1
 0.141031       1
-0.752625       1
Name: count, Length: 1560, dtype: int64

Model Selection

In [8]:
x = df.drop(columns=['species'])
y = df['species']

x_train,x_temp,y_train,y_temp=train_test_split(x,y,test_size=0.2,random_state=42)
x_test,x_val,y_test,y_val=train_test_split(x_temp,y_temp,test_size=0.5,random_state=42)

In [9]:
x_temp.shape, y_temp.shape

((1378, 12), (1378,))

In [10]:
x_test.shape, y_test.shape

((689, 12), (689,))

In [15]:
x_val.shape, y_val.shape

((689, 12), (689,))

In [14]:
y_train.head()

969                             White shark
6566                          a small shark
4338        Shark involvement not confirmed
5514    3 m to 4 m [10' to 13'] white shark
5985        Shark involvement not confirmed
Name: species, dtype: string

In [21]:
model = LinearRegression()
linear_model = model.fit(x_train, y_train)
linear_model

LinearRegression()

In [ ]:
#klib

kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(linear_model, x, y, cv=kf, scoring='neg_mean_squared_error')
cv_scores = np.sqrt(-cv_scores)

print("Cross validation score: ", cv_scores)
print("Mean Cv score: ", np.mean(cv_scores))



Cross validation score:  [0.98421409 0.96195885 1.02077924 0.94580631 0.96763686]
Mean Cv score:  0.9760790700256681


In [59]:
type(cv_scores)

numpy.ndarray

In [ ]:
#klib

y_pred = linear_model.predict(x_test)
mse_linear = mean_squared_error(y_pred, y_test)
r2_linear = r2_score(y_pred, y_test)

print(mse_linear)
print(r2_linear)

0.9628090319210653
-17.24502642694796


In [75]:
data = [
  ["mse_linear", mse_linear],
  ["r2_linear", r2_linear],
  ["mean cv score", np.mean(cv_scores)]
]

for i, score in enumerate(cv_scores, 1):
    data.append([f"cv_score_{i}", score])

headers = ['Method', 'Result']

In [76]:
table = tabulate(data, headers, tablefmt='grid')

Result with Klib

In [77]:
print(table)

+---------------+------------+
| Method        |     Result |
+===============+============+
| mse_linear    |   0.962809 |
+---------------+------------+
| r2_linear     | -17.245    |
+---------------+------------+
| mean cv score |   0.976079 |
+---------------+------------+
| cv_score_1    |   0.984214 |
+---------------+------------+
| cv_score_2    |   0.961959 |
+---------------+------------+
| cv_score_3    |   1.02078  |
+---------------+------------+
| cv_score_4    |   0.945806 |
+---------------+------------+
| cv_score_5    |   0.967637 |
+---------------+------------+


Result wihtout Klib

In [22]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(linear_model, x, y, cv=kf, scoring='neg_mean_squared_error')
cv_scores = np.sqrt(-cv_scores)

print("Cross validation score: ", cv_scores)
print("Mean Cv score: ", np.mean(cv_scores))

Cross validation score:  [0.96626364 0.96634035 0.9803697  0.9642037  0.97956329]
Mean Cv score:  0.9713481344540016


In [23]:

y_pred = linear_model.predict(x_test)
mse_linear = mean_squared_error(y_pred, y_test)
r2_linear = r2_score(y_pred, y_test)

print(mse_linear)
print(r2_linear)

0.918120455454211
-13.254661045237798


In [31]:
nk_data = [
  ["mse_linear", mse_linear],
  ["r2_linear", r2_linear],
  ["mean cv score", np.mean(cv_scores)]
]

for i, score in enumerate(cv_scores, 1):
    nk_data.append([f"cv_score_{i}", score])

nk_headers = ['Method', 'Result']
nk_table = tabulate(nk_data, nk_headers, tablefmt='grid')
print(nk_table)

+---------------+------------+
| Method        |     Result |
+===============+============+
| mse_linear    |   0.91812  |
+---------------+------------+
| r2_linear     | -13.2547   |
+---------------+------------+
| mean cv score |   0.971348 |
+---------------+------------+
| cv_score_1    |   0.966264 |
+---------------+------------+
| cv_score_2    |   0.96634  |
+---------------+------------+
| cv_score_3    |   0.98037  |
+---------------+------------+
| cv_score_4    |   0.964204 |
+---------------+------------+
| cv_score_5    |   0.979563 |
+---------------+------------+
